In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Digraph
%matplotlib inline

# Biological Neuron
Impulses come from the dendrites through the cell body and this is transformed to a response

![neuron](img/neuron.png)

# Perceptron
A perceptron tries to model a biological neuron where x(i)=dendrite
The cell body sums and transforms the input into a response using an activation function like `tanh` or `relu`.

![perceptron](img/perceptron.jpeg)

# Objective
Can we write a perceptron from scratch?

In [ ]:
# Squishes any range of inputs between -1, 1
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2)))
plt.grid()

In [ ]:
def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
# Defining a Value class to do math functions and store representations.
class Value_manual:
    def __init__(self, data, _children=(), _op="", label=""):
        """
        param data: Stores the value
        param _children: Stores the values that gave rise to self.data
        param _op: Math operation between children that produced self.data
        """
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value=({self.data})"
        
    def __add__(self, other):
        out = Value_manual(self.data + other.data, (self, other), _op="+")
        return out
        
    def __mul__(self, other):
        out = Value_manual(self.data * other.data, (self, other), _op="*")
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1)/(math.exp(2 * x) + 1)
        out = Value_manual(data=t, _children=(self,), _op="tanh")
        return out

In [ ]:
# Defining a 2 input neuron
x1 = Value_manual(2.0, label="x1")
w1 = Value_manual(-3.0, label="w1")
x2 = Value_manual(0.0, label="x2")
w2 = Value_manual(1.0, label="w2")
x1w1 = x1 * w1; x1w1.label="x1w1"
x2w2 = x2 * w2; x2w2.label="x2w2"
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1w1x2w2"
b = Value_manual(6.8813735870195432, label="b")
n = x1w1x2w2 + b; n.label="n"
o = n.tanh(); o.label="o"

In [ ]:
# Manual backprop
o.grad = 1.0
# do/dn = 1 - tanh(n)**2
n.grad = 1 - o.data**2
b.grad = n.grad
x1w1x2w2.grad = n.grad
x1w1.grad = x1w1x2w2.grad
x2w2.grad = x1w1x2w2.grad
w1.grad = x1.data * x1w1.grad
x1.grad = w1.data * x1w1.grad
w2.grad = x2.data * x1w1.grad
x2.grad = w2.data * x1w1.grad

In [ ]:
draw_dot(o)

# Rewriting `Value` to do automatic backpropogation

In [ ]:
# Defining a Value class to do math functions and store representations.
class Value_semi_manual:
    def __init__(self, data, _children=(), _op="", label=""):
        """
        param data: Stores the value
        param _children: Stores the values that gave rise to self.data
        param _op: Math operation between children that produced self.data
        """
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0
        # Every operation will have its own backprop.
        self._backward = lambda: None
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value=({self.data})"
        
    def __add__(self, other):
        out = Value_semi_manual(self.data + other.data, (self, other), _op="+")
        # chaining the current `.grad` based on the out.grad value.
        # This is the reason why we initiate o.grad = 1.
        # That chains `out.grad` `_backward`s through all the `_ops` and `Value`s.
        def _backward():
            self.grad = 1 * out.grad
            other.grad = 1 * out.grad            
        out._backward = _backward
        
        return out
        
    def __mul__(self, other):
        out = Value_semi_manual(self.data * other.data, (self, other), _op="*")
        
        def _backward():
            self.grad = other.data * out.grad
            other.grad = self.data * out.grad
        out._backward = _backward
        
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1)/(math.exp(2 * x) + 1)
        out = Value_semi_manual(t, (self,), "tanh")

        def _backward():
            self.grad = (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

In [ ]:
# Defining a 2 input neuron
x1 = Value_semi_manual(2.0, label="x1")
w1 = Value_semi_manual(-3.0, label="w1")
x2 = Value_semi_manual(0.0, label="x2")
w2 = Value_semi_manual(1.0, label="w2")
b = Value_semi_manual(6.8813735870195432, label="b")
x1w1 = x1 * w1; x1w1.label="x1w1"
x2w2 = x2 * w2; x2w2.label="x2w2"
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1w1x2w2"
n = x1w1x2w2 + b; n.label="n"
o = n.tanh(); o.label="o"

In [ ]:
# Backprop with the _backward() methods
o.grad = 1.0
o._backward()
n._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()

In [ ]:
draw_dot(o)

In [ ]:
# Defining a Value class to do math functions and store representations.
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        """
        param data: Stores the value
        param _children: Stores the values that gave rise to self.data
        param _op: Math operation between children that produced self.data
        """
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0
        # Every operation will have its own backprop.
        self._backward = lambda: None
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value=({self.data})"
        
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), _op="+")
        # chaining the current `.grad` based on the out.grad value.
        # This is the reason why we initiate o.grad = 1.
        # That chains `out.grad` `_backward`s through all the `_ops` and `Value`s.
        def _backward():
            self.grad += 1 * out.grad
            other.grad += 1 * out.grad            
        out._backward = _backward
        
        return out
        
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), _op="*")
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1)/(math.exp(2 * x) + 1)
        out = Value(t, (self,), "tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        # Automating the manual _backward() calls using topological sort.
        # Sort the DAG such that o is last.
        # Call _backward on the queue.
        topo = []
        visited = set()
        def build_topo(v):
          if v not in visited:
            visited.add(v)
            for child in v._prev:
              build_topo(child)
            topo.append(v)
        build_topo(self)
        self.grad = 1.0        
        for node in reversed(topo):
            node._backward()

In [ ]:
# Defining a 2 input neuron
x1 = Value(2.0, label="x1")
w1 = Value(-3.0, label="w1")
x2 = Value(0.0, label="x2")
w2 = Value(1.0, label="w2")
b = Value(6.8813735870195432, label="b")
x1w1 = x1 * w1; x1w1.label="x1w1"
x2w2 = x2 * w2; x2w2.label="x2w2"
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label="x1w1x2w2"
n = x1w1x2w2 + b; n.label="n"
o = n.tanh(); o.label="o"

In [ ]:
draw_dot(o)

In [ ]:
o.backward() #voila!

In the case of repeat variables we want to accumulate gradients in the `Value` class.

So changing `self.grad = ... * out.grad` to `self.grad += ... * out.grad`

In [ ]:
# Without doing the += in the cell above this will give the wrong gradient for `a`
a = Value(3, label="a")
b = a + a; b.label = "b"
b.backward()
draw_dot(b)